A few years ago, I was talking with NBA and Analytics guru and he asked me this problem:

<br>
_Let's say there is a team who you expect to win **75%** of its games in a given **82-game** NBA regular season (and the probability of winning each game = 75%).  
If you wanted to determine the probability that the team will never lose consecutive games at any point during this 82-game season?_
<br>

Let us try to answer this question. We will used two methods. First, we will calculate the exact probability, then we will run a simulation to verify our first solution.

In [24]:
import scipy.misc
import scipy.stats
import numpy as np
import random
import pandas as pd
import time

### Approach 1 - Stats

Let us look with a simplified case.

-  Let us say team is going to win exactly 75% of its game.
    -  it means 62 Wins and 20 losses
    -  This probaility is combin(82,62) $*$ (0.75)^(62) $*$ (0.25)^(82-62)
-  I want to get how many combinations are possible with zero back-to-back losses. 
    -  I need to have “? W ? W ?.... ? W ?”. 
    -  I have 62 W, and I know that I can put the 20 losses whenever I want in between these 62 W. 
    -  Thus, I have 63 available spots for 20 L. The corresponding number of combinations is combin (63,20).
- What is the total number of combinations?
    -  82 games and have 20 losses
- The probability of having 20 losses with no consecutives losses in a 82 game
    -  combin (63,20) / combin (82,20)
    
We just have to loop from 0 to 82 wins during a regular season and get the corresponding probabilities.

In [59]:
def stat_method(number_games, proba_win):
    """ stat_method
    number_games = number of games played in a season
    proba win = proba win each game """
    proba_tot_no_b2b = 0
    
    for x in range(number_games+1):
        proba_wins = scipy.stats.binom.pmf(x, number_games, proba_win)
        proba_no_b2b = scipy.misc.comb(x+1, number_games-x) / scipy.misc.comb(number_games, number_games-x)
        proba_case_no_b2b = proba_wins * proba_no_b2b
        proba_tot_no_b2b += proba_case_no_b2b

    return ("{0:.3f}%".format(proba_tot_no_b2b*100))

In [60]:
print(stat_method(82, 0.75))

1.291%


### Approach 2 - Simulations

Let us look with a simplified case

-  A season consists of 82 games - probability of winning each game is 0.75
    -  I can simulation the output of the 82 games
    -  **1 for a win, 0 for a loss**
    - **82** _Bernouilli trials_ with a probability of success equals to **0.75**
-  Let us look at our season
    - Is there any _back to back losses_? 
    - If No, count 1, else count 0
    
We can _simulatate_ **10^6** seasons, and look how often we have season without back to back losses

In [69]:
def simul_method(number_games, proba_win, number_season):
    """ simul_method
    number_games = number of games played in a season
    proba win = proba win each game
    number season = number season (the more the merrier) """
    count = 0
    for k in range(number_season):
        season = np.random.binomial(1, proba_win, number_games)
        if not any((i==0 and j==0) for i,j in zip(season, season[1:])):
            count +=1
    return ("{0:.3}%".format((count / number_season)*100))

In [72]:
print(simul_method(82, 0.75, 1000000))

1.28%


As a conclusion, we see that both solutions are almost identical. 

- The pure stat approach is going to give the exact answer, but it is not trivial to model nor understand.
- The simulation provides a very good estimation and is easier to understand